In [1]:
import re
import pandas as pd

In [2]:
## Open and Encode to utf-8
f = open("ce-b-unofficial-only-chat.txt", 'r', encoding='utf-8')

In [3]:
data = f.read()

In [4]:
print(data)

11/04/22, 10:51 am - Harshil GECG created group "CE-B unofficial only🤩"
11/04/22, 10:51 am - You were added
28/10/23, 11:04 am - +91 70484 03967: <Media omitted>
28/10/23, 11:36 pm - Your security code with Vardhan Raina GECG changed. Tap to learn more.
30/10/23, 11:57 am - +91 93278 41812: CPDP students have to speak one quote with meaning for viva,
Message by NMR sir
30/10/23, 12:06 pm - Sujal Padhara GECG: Sem 6 Elective Choice 2023 form
Instructions:
1) All CE Sem 5 students have to fill the elective choice form on or before 5th Nov 2023.
2) Carefully go through the GTU syllabus before filling the elective choice form.
3) Choice once made will not be altered.

https://forms.gle/LERZapUsf9f1k3uB8
30/10/23, 12:09 pm - +91 84018 88762: <Media omitted>
30/10/23, 12:09 pm - +91 84018 88762: CE B1 batch dates for Software engineering subject
30/10/23, 3:24 pm - Umang Limbasiya GECG: SE ma fail thya hoy ane GTU paper lakhvana hoy ene 3/11/2023 e 11:00 AM e MHV mam ne batavi devu......
31/

In [5]:
print(type(data))

<class 'str'>


In [6]:
## regex for time and message extraction
messagePattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\spm\s-\s|\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\sam\s-\s'

In [7]:
messages = re.split(messagePattern, data)[1:]
len(messages)

440

In [8]:
datesPattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\spm|\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\sam'

In [9]:
dates = re.findall(datesPattern, data)
len(dates)

440

In [10]:
## Conversion in DataFrame
df = pd.DataFrame({'user_message': messages, 'date': dates})

## Convert Time
df['date'] = pd.to_datetime(df['date'])


df.head()

C:\Users\conta\AppData\Local\Temp\ipykernel_26044\4215724267.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,user_message,date
0,"Harshil GECG created group ""CE-B unofficial on...",2022-11-04 10:51:00
1,You were added\n,2022-11-04 10:51:00
2,+91 70484 03967: <Media omitted>\n,2023-10-28 11:04:00
3,Your security code with Vardhan Raina GECG cha...,2023-10-28 23:36:00
4,+91 93278 41812: CPDP students have to speak o...,2023-10-30 11:57:00


In [11]:
# Separate Users and Messages
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]:
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])

df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df.head()


,date,user,message
0,2022-11-04 10:51:00,group_notification,"Harshil GECG created group ""CE-B unofficial on..."
1,2022-11-04 10:51:00,group_notification,You were added\n
2,2023-10-28 11:04:00,+91 70484 03967,<Media omitted>\n
3,2023-10-28 23:36:00,group_notification,Your security code with Vardhan Raina GECG cha...
4,2023-10-30 11:57:00,+91 93278 41812,CPDP students have to speak one quote with mea...


In [12]:
df['day'] = df['date'].dt.day
df['day_name'] = df['date'].dt.day_name()
df['month'] = df['date'].dt.month_name()
df['year'] = df['date'].dt.year
df.loc[df['date'].dt.hour <= 12, 'hour'] = df['date'].dt.hour
df.loc[df['date'].dt.hour > 12, 'hour'] = df['date'].dt.hour - 12
df['hour'] = df['hour'].astype(int)
df['minute'] = df['date'].dt.minute
df.loc[df['date'].dt.hour <= 11, 'meridiem'] = 'AM' 
df.loc[df['date'].dt.hour >  11, 'meridiem'] = 'PM'
df.loc[df['date'].dt.hour ==  0, 'hour'] = 12

period = []
for i in range(df.shape[0]):
    if (df.iloc[i]["hour"] == 11) & (df.iloc[i]["meridiem"] == "PM"):
        period.append(str(df.iloc[i]["hour"]) + " PM" + " - " + str(df.iloc[i]["hour"] + 1) + " AM")
    elif (df.iloc[i]["hour"] == 11) & (df.iloc[i]["meridiem"] == "AM"):
        period.append(str(df.iloc[i]["hour"]) + " AM" + " - " + str(df.iloc[i]["hour"] + 1) + " PM")
    elif (df.iloc[i]["hour"] == 12):
        period.append(str(df.iloc[i]["hour"]) + " " + str(df.iloc[i]["meridiem"]) + " - " + "1" + " " + str(df.iloc[i]["meridiem"]))
    else:
        period.append(str(df.iloc[i]["hour"]) + " " + str(df.iloc[i]["meridiem"]) + " - " + str(df.iloc[i]["hour"] + 1) + " " + str(df.iloc[i]["meridiem"]))
        
df["period"] = period


In [13]:
df

,date,user,message,day,day_name,month,year,hour,minute,meridiem,period
0,2022-11-04 10:51:00,group_notification,"Harshil GECG created group ""CE-B unofficial on...",4,Friday,November,2022,10,51,AM,10 AM - 11 AM
1,2022-11-04 10:51:00,group_notification,You were added\n,4,Friday,November,2022,10,51,AM,10 AM - 11 AM
2,2023-10-28 11:04:00,+91 70484 03967,<Media omitted>\n,28,Saturday,October,2023,11,4,AM,11 AM - 12 PM
3,2023-10-28 23:36:00,group_notification,Your security code with Vardhan Raina GECG cha...,28,Saturday,October,2023,11,36,PM,11 PM - 12 AM
4,2023-10-30 11:57:00,+91 93278 41812,CPDP students have to speak one quote with mea...,30,Monday,October,2023,11,57,AM,11 AM - 12 PM
...,...,...,...,...,...,...,...,...,...,...,...
435,2024-04-03 15:19:00,Krushang GECG,This message was deleted\n,3,Wednesday,April,2024,3,19,PM,3 PM - 4 PM
436,2024-04-03 17:06:00,Naman GECG,"If anyone has TOC Tutorial questions, Then ple...",3,Wednesday,April,2024,5,6,PM,5 PM - 6 PM
437,2024-04-03 17:08:00,Krushang GECG,,3,Wednesday,April,2024,5,8,PM,5 PM - 6 PM
438,2024-04-03 17:11:00,+91 94261 41533,RIP 😔💐\n,3,Wednesday,April,2024,5,11,PM,5 PM - 6 PM
